In [ ]:
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.optimizers import SGD
import socket
import struct
import pandas as pd
import numpy as np
from collections import deque
import random

### Train from csv

In [ ]:
lst = [[1.0, 0, 0], [0, 1.0, 0], [0, 0, 1.0]]

df = pd.read_csv("data.csv")
data = np.array(df)

x = data[:, :6700]
y = data[:, 6700]

In [ ]:
x = x.reshape(54202, 6700)
y = np.array([lst[int(i)] for i in y])

In [ ]:
model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(6700, )))
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))

epochs = 25
lr = 0.02
decay = lr / epochs
sgd = SGD(lr=lr, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(x, y, batch_size=32, epochs=epochs)
model.save("model.h5")

In [ ]:
%%timeit
np.argmax(model.predict_on_batch(np.random.rand(1, 6700)))

### Train from realtime

In [ ]:
class Env:
    def __init__(self):
        self.UDP_IP = "127.0.0.1"
        self.UDP_PORT_RECV = 9003
        self.UDP_PORT_SEND = 9004

        self.send = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.recv = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.recv.bind((self.UDP_IP, self.UDP_PORT_RECV))

    def reset(self):
        self.send.sendto(struct.pack("i", -1), (self.UDP_IP, self.UDP_PORT_SEND))
        data, _ = self.recv.recvfrom(26808)
        data = np.array(struct.unpack('6702f', data))[:6700]
        return data.reshape(1, 6700)

    def step(self, action):
        self.send.sendto(struct.pack("i", action), (self.UDP_IP, self.UDP_PORT_SEND))
        data, _ = self.recv.recvfrom(26808)
        data = np.array(struct.unpack('6702f', data))
        new_state = data[:6700].reshape(1, 6700)
        reward = data[6700]
        lose = True if data[6701] > 0 else False
        return new_state, reward, lose

env = Env()

In [ ]:
class Train:
    def __init__(self, env):
        self.env = env
        self.gamma = 0.99
        self.epsilon = 1
        self.epsilon_decay = 0.05
        self.epsilon_min = 0.01
        lr = 0.001
        decay = 0.01 / 25
        self.sgd = SGD(lr=lr, momentum=0.9, decay=decay, nesterov=False)
        self.replay_buffer = deque(maxlen=20000)
        self.train_network = self.create_model()
        self.episode_num = 400
        self.num_pick_from_buffer = 32
        self.target_network = self.create_model()
        self.target_network.set_weights(self.train_network.get_weights())

    def create_model(self):
        model = Sequential()
        model.add(Dense(128, activation='relu', input_shape=(6700, )))
        model.add(Dense(128, activation='relu', input_shape=(6700, )))
        model.add(Dense(3, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer=self.sgd, metrics=['accuracy'])
        return model

    def get_best_action(self, state):
        self.epsilon = max(self.epsilon_min, self.epsilon)
        if np.random.rand(1) < self.epsilon:
            action = np.random.randint(0, 3)
        else:
            action = np.argmax(self.train_network.predict(state)[0])
        return action

    def train_from_buffer(self):
        if len(self.replay_buffer) < self.num_pick_from_buffer:
            return
        samples = random.sample(self.replay_buffer, self.num_pick_from_buffer)
        states = []
        new_states = []
        for sample in samples:
            state, action, reward, new_state, done = sample
            states.append(state)
            new_states.append(new_state)

        new_array = np.array(states)
        states = new_array.reshape(self.num_pick_from_buffer, 6700)
        new_array2 = np.array(new_states)
        new_states = new_array2.reshape(self.num_pick_from_buffer, 6700)
        targets = self.train_network.predict(states)
        new_state_targets = self.target_network.predict(new_states)

        i = 0
        for sample in samples:
            state, action, reward, new_state, done = sample
            target = targets[i]
            if done:
                target[action] = reward
            else:
                Q_future = max(new_state_targets[i])
                target[action] = reward + Q_future * self.gamma
            i += 1

        self.train_network.fit(states, targets, epochs=1, verbose=0)

    def original_try(self, current_state, eps):
        reward_sum = 0
        iteration = 0

        while True:
            best_action = self.get_best_action(current_state)
            new_state, reward, lose = self.env.step(best_action)
            done = False
            if reward > 0.8:
                done = True
            if iteration > 30:
                self.replay_buffer.append([current_state, best_action, reward, new_state, done])
            self.train_from_buffer()
            reward_sum += reward
            current_state = new_state
            
            iteration +=1
            
            if lose:
                break

        self.target_network.set_weights(self.train_network.get_weights())

        print(f"ep: {eps}, epsilon: {self.epsilon}, iteration: {iteration}, reward: {reward_sum}")
        self.epsilon -= self.epsilon_decay
        if self.epsilon < self.epsilon_min:
            self.epsilon = self.epsilon_min

    def start(self):
        for eps in range(self.episode_num):
            current_state = self.env.reset()
            self.original_try(current_state, eps)


train = Train(env)

In [ ]:
train.start()

In [ ]:
train.train_network.save("model.h5")

In [ ]:
env.send.close()
env.recv.close()

### Predict

In [ ]:
model = load_model("model.h5")

UDP_IP = "127.0.0.1"
UDP_PORT_RECV = 9003
UDP_PORT_SEND = 9004


send = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
recv = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
recv.bind((UDP_IP, UDP_PORT_RECV))

In [ ]:
send.sendto(struct.pack("i", -1), (UDP_IP, UDP_PORT_SEND))

while True:
    data, addr = recv.recvfrom(26808)
    print('x')
    inp = struct.unpack('6702f', data)[:6700]
    inp = np.array(inp).reshape(1, 6700)
    ret = model.predict(inp)
    ret = np.argmax(ret[0])
    send.sendto(struct.pack("i", ret), (UDP_IP, UDP_PORT_SEND))